In [1]:
import os
import sys
import pandas as pd
import numpy as np

from pygus.gus.population import generate_population_allometrics

In [2]:
def percentage_change(current_value, initial_value):
    """The function to copmpute percentage change with respect to a given base value."""
    if initial_value == 0:
        return 0

    return ((current_value - initial_value) / initial_value) * 100


def compute_under_canopy_area_m2(crown_diameter):
    return pow((crown_diameter / 2), 2) * np.pi


def compute_leaf_area_index(
    dbh,
    tree_height,
    crown_depth,
    crown_diameter,
    crown_missing=0,
    shade_factor=0.85,
):
    """The function given allometrics of a tree computes its leaf, bark and plant area indices.

    Args:
        dbh: (:obj:`float`): the diameter in cm of the trunk usually measured at 1.3m from the ground.
        tree_height: (:obj:`float`): The tree height in meters.
        crown_debth: (:obj:`float`): The vertical length of tree crown in meters.
        crown_diameter: (:obj:`float`): The horizontal length (diameter) of tree crown in meters.
        crown_missing: (:obj:`float`): The percentage loss of the crown.
        shade_factor: (:obj:`float`): The percentage of sky covered by foliage and branches.

        Shade factor, is the percentage of sky covered by foliage and branches within the perimeter of individual tree crowns,
        can vary by species from about 60% to 95% when trees are in-leaf (McPherson, 1984). The value below is set according to
        Glasgow mean and needs to be updated per city or per tree.

    Returns:
        (:obj:`float`): It eturns the LAI.
    Note:
        The beta multipliers and the main equation is based on Nowak (1996).

    TODO:
        Parametrize beta multipliers.
    """
    loss = crown_missing
    th = tree_height
    cw = crown_diameter
    cd = crown_depth
    sf = shade_factor
    beta_0 = -4.3309
    beta_1 = 0.2942
    beta_2 = 0.7312
    beta_3 = 5.7217
    beta_4 = 0.0148

    # Outer surface area estimate below is based on Gacka-Grzesikiewicz (1980).
    under_canopy = compute_under_canopy_area_m2(cw)
    crown_surface = np.pi * cw * (cd + cw) / 2
    leaf_area = (1 - loss) * np.exp(
        beta_0 + beta_1 * th + beta_2 * cw + beta_3 * sf - beta_4 * crown_surface
    )
    leaf_area_index = leaf_area / under_canopy  # m^2/m^2
    return leaf_area_index  # An intuitive way to think about LAI is as the one-sided green leaf area per unit ground surface area. !!


def compute_gross_canopy_index(tree_height, crown_depth, crown_diameter):
    """The function given allometrics of a tree computes an index to account shading effect immediately underneath its canopy.

    Args:
        tree_height: (:obj:`float`): The tree height in meters.
        crown_height: (:obj:`float`): The vertical length of tree crown in meters.
        crown_width: (:obj:`float`): The horizontal length (diameter) of tree crown in meters.

    Returns:
        (:obj:`float`): returns the GCI index
    Note:
        The index is based on (Zhang et.al., 2020). It accounts for the overall immediate shading effect of a tree.
        It accounts both crown diameter (tw) and the crown base height (th - ch), the distance from the ground to the start of
        the canopy.
    """
    th = tree_height
    cw = crown_diameter
    cd = crown_depth
    crown_base_height = th - cd
    if crown_base_height < 0.1:
        crown_base_height = 0.1
    GCI = cw / crown_base_height
    return GCI  # GCI is just a ratio of crown diameter to crown base height. It is a measure of the overall immediate shading effect of a tree.


def compute_crown_volume_index(leaf_area_index, crown_canopy_index):
    """The function given computes a composite index taking into account both leaf density and canopy shape.

    Args:
        leaf_area_indext: (:obj:`float`): The index on the leaf density per the area under the canopy.
        crown_canopy_index: (:obj:`float`): The index that accounts for the geomtric shape of the canopy relevant to shading.

    Returns:
        (:obj:`float`): returns the CVI index
    Note:
        The index is adopted from (Zhang et.al., 2020).
    """
    # CVI is a composite index taking into account both leaf density and canopy shape.
    return leaf_area_index * crown_canopy_index


dT_max = 0.67
m1 = 0.1
m2 = 0.75
m4 = 0.75
m5 = 0.2
T0 = 10
T1 = 18
T2 = 26
T3 = 35
T4 = 45
T5 = 50
avg_conifer_cooling_factor = 0.66


def cooling_multiplier(T):
    if T < T0:
        return 0
    if T < T1:
        return m1 * dT_max
    if T < T2:
        return m2 * dT_max
    if T < T3:
        return dT_max
    if T < T4:
        return m4 * dT_max
    if T < T5:
        return m5 * dT_max
    return 0


# Comparing this index of the canopy, and how it changes across a timespan from A to B
# We can estimate the further cooling potential of that tree canopy in B relative to A (in degrees C)
def compute_cooling(canopy_change_cvi, T=30, species=None):
    # The CVI is a composite index taking into account both leaf density and canopy shape. The higher the CVI the greater the cooling effect.
    if species == "conifer" or species == "miyawaki_conifer":
        species_factor = 0.6
    elif species == "decidu" or species == "miyawaki_decidu":
        species_factor = 0.9
    else:
        species_factor = 1
    return species_factor * np.sqrt(canopy_change_cvi / 10) * cooling_multiplier(T)


def calculate_nth_year_CVI_and_percentage_canopy_cover(df):
    nth_CVIs = df.apply(
        lambda row: compute_crown_volume_index(
            compute_leaf_area_index(
                row["dbh"], row["height"], row["crownH"], row["crownW"]
            ),
            compute_gross_canopy_index(row["height"], row["crownH"], row["crownW"]),
        ),
        axis=1,
    )

    # Same for the coverage in year N
    canopy_cover_N = (
        df.apply(
            lambda row: compute_under_canopy_area_m2(row["crownW"]), axis=1
        ).sum()
    )
    
    return nth_CVIs, canopy_cover_N

# Heritage Forest
180 Sqm

632 saplings

26 species

**Canopy 10% 63**
1. Lime - small 21
2. Oak Sessile 21
3. Oak English 21

**Tree 30% 190**
1. Hornbeam 34
2. Willow goat 26
3. Willow bay 26 
4. Rowan 26
5. Maple 26
6. Crab apple 26
7. Willow - grey 26

**Sub Tree 30% 190**
1. Yew 26
2. Holly 26
3. Hazel 34
4. Hawthorn 26
5. Blackthorn 26
6. Whitebeam 26
7. Willow wooly 26

**Shrub 30% 189**

In [3]:
lng_lat = [-0.161336, 51.496822]
population_size = 632 - 189 # minus shrubs
area_m2 = 180 #m2
density_per_ha = 3.2 * 10000 #trees/m2 * 10000m2/ha
dbh_range_saplings = [0.2,0.4] #cm
height_range_saplings = [0.2,0.4] #m
crownW_range_saplings = [0.02, 0.03] #m
tree_families = {
    'Lime - small': 'miyawaki_decidu',
    'Oak Sessile': 'miyawaki_decidu',
    'Oak English': 'miyawaki_decidu',
    'Hornbeam': 'miyawaki_decidu',
    'Willow goat': 'miyawaki_decidu',
    'Willow bay': 'miyawaki_decidu',
    'Rowan': 'miyawaki_decidu',
    'Maple': 'miyawaki_decidu',
    'Crab apple': 'miyawaki_decidu',
    'Willow - grey': 'miyawaki_decidu',
    'Yew': 'miyawaki_conifer',
    'Holly': 'miyawaki_decidu',  # Evergreen but not a conifer, it's broadleaved, so I chose miyawaki_decidu
    'Hazel': 'miyawaki_decidu',
    'Hawthorn': 'miyawaki_decidu',
    'Blackthorn': 'miyawaki_decidu',
    'Whitebeam': 'miyawaki_decidu',
    'Willow wooly': 'miyawaki_decidu'
}
tree_data = {
    'Lime - small': 21,
    'Oak Sessile': 21,
    'Oak English': 21,
    'Hornbeam': 34,
    'Willow goat': 26,
    'Willow bay': 26,
    'Rowan': 26,
    'Maple': 26,
    'Crab apple': 26,
    'Willow - grey': 26,
    'Yew': 26,
    'Holly': 26,
    'Hazel': 34,
    'Hawthorn': 26,
    'Blackthorn': 26,
    'Whitebeam': 26,
    'Willow wooly': 26
}

id = 0
data = pd.DataFrame(columns=['id', 'species'])
data.set_index('id', inplace=True)
for key in tree_data:
    # add that number of trees to the dataset
    for i in range(tree_data[key]):
        data.loc[id] = {'species': key}
        id += 1

# classify each of the species as miyawaki_decidu or coniferous
data['species'] = data['species'].map(tree_families)
conifer_ratio = len(data[data['species'] == 'conifer']) / len(data)
print(conifer_ratio)
df = generate_population_allometrics(data, lng_lat, area_m2, dbh_range_saplings, height_range_saplings, crownW_range_saplings, conifer_ratio)

print(df.tail(20))

0.0
      id          species        lat       lng  xpos  ypos  condition    dbh  \
423  423  miyawaki_decidu  51.496782 -0.161314     6     2       good  0.227   
424  424  miyawaki_decidu  51.496847 -0.161290     8     9       good  0.216   
425  425  miyawaki_decidu  51.496767 -0.161341     4     0  excellent  0.274   
426  426  miyawaki_decidu  51.496862 -0.161288     8    11       fair  0.346   
427  427  miyawaki_decidu  51.496881 -0.161296     7    13  excellent  0.279   
428  428  miyawaki_decidu  51.496788 -0.161383     1     2       good  0.280   
429  429  miyawaki_decidu  51.496838 -0.161297     7     8       good  0.202   
430  430  miyawaki_decidu  51.496829 -0.161361     3     7  excellent  0.245   
431  431  miyawaki_decidu  51.496811 -0.161286     8     5       fair  0.316   
432  432  miyawaki_decidu  51.496766 -0.161295     8     0  excellent  0.333   
433  433  miyawaki_decidu  51.496837 -0.161277     9     8  excellent  0.247   
434  434  miyawaki_decidu  51.496807

Forest Report: 1 Year

DATE: 12.10.2022

Survival Rate: 97%

Average of Tallest 3 Trees: 170cm

This little forest is vibrant and adapting well to urban life. The summer heatwave has had little negative effect and the saplings are maturing well. Shade-tolerant species such as dogwood (Cornus sanguinea), spindle (Euonymus europaeus) and hazel (Corylus avellana) are growing well in the dense shade without becoming leggy - often an issue with growth in shady areas.

The rowan (Sorbus aucuparia) and crack willow (Salix fragilis) are currently both the tallest species, and the crack willow has a girth of 25mm. Crack willow is interesting in that it's the only variety of willow that grows well in drier conditions - willow is normally associated with damper conditions. Crack willow is also a fantastic early source of forage for pollinators when the catkins arrive in early spring. It is promising to see this species thriving.

The oaks too are doing well too - planted as the climax species to eventually take over from the current canopy of London plane trees.

Heritage Forest has really transformed this pocket of Kensington and Chelsea, bringing some much needed greenery to an area dominated by buildings and traffic.

In [4]:
import folium

# Create a folium map object
m = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], zoom_start=18)

# Plot points
for index, row in df.iterrows():
    folium.Marker([row['lat'], row['lng']], popup="{},{}".format(row['lat'], row['lng'])).add_to(m)

# Save or show map
# m.save("sample.html")

In [5]:
import json
import pandas as pd
from pygus.gus.models import Urban, SiteConfig, WeatherConfig

scenario_file = "../../pygus/gus/inputs/scenario.json"
london_weather = WeatherConfig(138, 9)
site_config = SiteConfig(
    total_m2=area_m2,
    impervious_m2=0,
    pervious_m2=area_m2,
    weather=london_weather,
    tree_density_per_ha=population_size/area_m2*10000,
    project_site_type="pocket"
)
try:
    scen = open(scenario_file)
except IOError as e:
    print(str(e))
scenario = json.loads(scen.read())
model = Urban(
    population=df,
    species_allometrics_file="../../pygus/gus/inputs/allometrics.json",
    site_config=site_config,
    scenario=scenario,
)

# model.run(scenario.get("time_horizon_years"))
impacts: pd.DataFrame = model.run()

Running for 25 steps
25 steps completed (pop. 443): 3.173398017883301


In [6]:
print(impacts.head())
agents = model.get_agent_data()
agents.reset_index(inplace=True)
agents.head(20)
agents.to_csv('agents_heritage_forest.csv')

      Storage         Seq   Avg_Seq  Released   Avg_Rel  Alive  Dead  \
0   15.261330   11.772589  0.027126  0.042096  0.000112    434     9   
1   40.962663   25.070000  0.057898  0.138046  0.000410    433    10   
2   89.426538   47.895732  0.110105  0.378232  0.001039    435     8   
3  159.167202   69.582142  0.161443  1.190171  0.002833    431    12   
4  262.147818  102.275774  0.235658  1.389511  0.003389    434     9   

   Critical  Dying  Poor  Replaced   Seq_std     Cum_Seq  
0         0      0     0         0  0.031696   11.772589  
1         0      0     0         9  0.079158   36.842589  
2         0      0     0        19  0.150878   84.738321  
3         0      0     0        25  0.218292  154.320463  
4         0      0     0        36  0.323813  256.596237  


In [15]:

print("Cooling effect of Heritage Forest (compared to 30C baseline):")

year_1_CVI, canopy_cover_1 = calculate_nth_year_CVI_and_percentage_canopy_cover(agents[agents['Step'] == 1])
print(canopy_cover_1)

for i in range(2, 11):
    year_i_CVI, canopy_cover_i = calculate_nth_year_CVI_and_percentage_canopy_cover(agents[agents['Step'] == i])
    cooling_year_i = compute_cooling(dCVI, T=30, species='miyawaki_decidu')
    print("Year {} cooling: {}".format(i, cooling_year_i))

Cooling effect of Heritage Forest (compared to 30C baseline):
761.5387955456672
842.321251381836
761.5387955456672
80.78245583616877
-12.57007364745401
Year 2 cooling: nan
940.875870404808
761.5387955456672
179.33707485914078
-16.34137600464103
Year 3 cooling: nan
1036.025629224173
761.5387955456672
274.4868336785057
-17.396308230889723
Year 4 cooling: nan
1154.4827986221403
761.5387955456672
392.9440030764731
-18.009006434231566
Year 5 cooling: nan
1268.9755018506762
761.5387955456672
507.436706305009
-17.862844571685073
Year 6 cooling: nan
1401.4995128354255
761.5387955456672
639.9607172897583
-17.868002296776893
Year 7 cooling: nan
1525.6309162789253
761.5387955456672
764.0921207332581
-17.850575519053443
Year 8 cooling: nan
1656.466686539856
761.5387955456672
894.9278909941888
-17.876541980387167
Year 9 cooling: nan
1788.18801128663
761.5387955456672
1026.6492157409627
-17.523885409781656
Year 10 cooling: nan


/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_7191/2885023614.py:145: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi / 10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_7191/2885023614.py:145: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi / 10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_7191/2885023614.py:145: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi / 10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_7191/2885023614.py:145: RuntimeWarning: invalid value encountered in sqrt
  return species_factor * np.sqrt(canopy_change_cvi / 10) * cooling_multiplier(T)
/var/folders/sq/279ztff55tg817zzbvzd4p0c0000gn/T/ipykernel_7191/2885023614.py:145: RuntimeWarning: invalid value encountered in sqrt
  retur